# Ejercicio adicional de fin de semana - semana 2

Ahora utilice todo lo que ha aprendido en la Semana 2 para crear un prototipo completo para la pregunta/respuesta técnica que creó en el Ejercicio de la Semana 1.

Esto debería incluir una interfaz de usuario de Gradio, transmisión, uso del indicador del sistema para agregar experiencia y la capacidad de cambiar entre modelos. ¡Puntos de bonificación si puedes demostrar el uso de una herramienta!

Si se siente atrevido, vea si puede agregar entrada de audio para poder hablar con él y hacer que responda con audio. ChatGPT o Claude pueden ayudarte o envíame un correo electrónico si tienes preguntas.

Publicaré una solución completa aquí pronto, a menos que alguien me gane...

Hay tantas aplicaciones comerciales para esto, desde un tutor de idiomas hasta una solución de incorporación empresarial, una IA complementaria y un curso (¡como este!) No puedo esperar a ver los resultados.

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama
import time

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "Eres un asistente útil para una empresa. "
system_message += "Dé respuestas breves y corteses. "
system_message += "Sea siempre preciso. Si no sabes la respuesta, dilo."

In [4]:
# Funciòn de extensión teléfonica

ext_tele = {"andres alvarez": "1001", "beatriz bravo": "1002", "carlos caceres": "1003", "diana díaz": "1004", "eduardo estévez": "1005", "fernanda figueroa": "1006"}

def get_ext_telefono(nombre):
    print(f"Tool para conocer la extensión teléfonica del personal {nombre}")
    name = nombre.lower()
    return ext_tele.get(name, "Unknown")

In [5]:
# Función:

telefono_function = {
    "name": "get_ext_telefono",
    "description": "Obtén la extensión teléfonica del personal. Llame a esto siempre que necesite saber la extensión de teléfono, por ejemplo, cuando un cliente pregunte '¿Cual es la extensión teléfonica?'",
    "parameters": {
        "type": "object",
        "properties": {
            "nombre": {
                "type": "string",
                "description": "La extensión teléfonica del empleado que quiere comunicarse",
            },
        },
        "required": ["nombre"],
        "additionalProperties": False
    }
}

In [6]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": telefono_function}]

In [84]:
def chat(message, history, temp, model):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    if model == "Chatgpt":
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, temperature = temp)
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            response, nombre = handle_tool_call(message)
            messages.append(message)
            messages.append(response)
            response = openai.chat.completions.create(model=MODEL, messages=messages)
    
        return response.choices[0].message.content
    else:
        # La llamada de Tools de Ollama no funciona correctamente, trate por todos lados :(
        response = ollama.chat(model="llama3.1",messages=messages)
        return response['message']['content']    

In [33]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    nombre = arguments.get('nombre')
    ext = get_ext_telefono(nombre)
    response = {
        "role": "tool",
        "content": json.dumps({"nombre": nombre,"extensión": ext}),
        "tool_call_id": tool_call.id
    }
    return response, nombre

In [85]:
# Aplicando stream en Gradio
def slow_echo(message, history, temp, model):
    print ("Temp:" + str(temp))
    print ("Modelo:" + model)
    stream = chat(message, history, temp,model)
    for i in range(len(stream)):
        time.sleep(0.05)
        yield stream[: i + 1]
        

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("<center><h1>ChatBot en aprendizaje</h1></center>")
            gr.ChatInterface(
                slow_echo,
                type="messages",
                show_progress = True,
                additional_inputs = [
                    gr.Slider(0, 1,value = 0.6,step = 0.1,interactive = True,label = "Desliza para graduar la respuesta",),
                    gr.Dropdown(["Chatgpt", "Llama3.1"], label="Modelo", info="Seleccione el modelo")
                ]
            )
demo.launch()


* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


Temp:0.6
Modelo:Chatgpt
Tool para conocer la extensión teléfonica del personal Carlos Caceres
Temp:0.6
Modelo:Llama3.1
